In [157]:
import json
from sqlalchemy import create_engine
from datetime import date
import pandas as pd
import pymysql
pymysql.install_as_MySQLdb()

In [158]:
from config import sf_username, sf_password, sf_security_token
from config import remote_db_endpoint, remote_db_port
from config import remote_db_name, remote_db_user, remote_db_pwd

from simple_salesforce import Salesforce
sf = Salesforce(username=sf_username, password=sf_password, security_token=sf_security_token)

In [159]:
engine = create_engine(f"mysql://{remote_db_user}:{remote_db_pwd}@{remote_db_endpoint}:{remote_db_port}/{remote_db_name}")
conn = engine.connect()

# 1) Course Table

In [160]:
#COURSE
course_data_df = pd.read_sql("SELECT * FROM course", conn)
course_data_df.head(30)

,ID_Course,CourseCode,CourseName,CreditHours,BootCampCourse
0,1,BC-DATAVIZ,Data Visualization and Analytics,12,1
1,2,BC-WEBDEV,Full Stack Web Development,12,1
2,3,BC-UIUX,User Interface/User Experience,12,1
3,4,CIS-349,Introduction to Databases,5,0
4,5,CIS-405,Database Programming,5,0
5,6,CIS-438,Database Administration,5,0
6,7,CIS-430,Business Systems Programming I,5,0
7,8,CIS-435,Business Systems Programming II,5,0


In [161]:
course_data_df.rename(columns={
    'ID_Course':'ID_Course__c',
    'CourseCode':'Course_Code__c',
    'CourseName':'Course_Name__c',
    'CreditHours':'Credit_Hours__c',
    'BootCampCourse':'Boot_Camp_Course__c',    
}, inplace=True)
course_data_df.head()

,ID_Course__c,Course_Code__c,Course_Name__c,Credit_Hours__c,Boot_Camp_Course__c
0,1,BC-DATAVIZ,Data Visualization and Analytics,12,1
1,2,BC-WEBDEV,Full Stack Web Development,12,1
2,3,BC-UIUX,User Interface/User Experience,12,1
3,4,CIS-349,Introduction to Databases,5,0
4,5,CIS-405,Database Programming,5,0


In [162]:
course_data_records = course_data_df.to_dict('records')
course_data_records

[{'ID_Course__c': 1,
  'Course_Code__c': 'BC-DATAVIZ',
  'Course_Name__c': 'Data Visualization and Analytics',
  'Credit_Hours__c': 12,
  'Boot_Camp_Course__c': 1},
 {'ID_Course__c': 2,
  'Course_Code__c': 'BC-WEBDEV',
  'Course_Name__c': 'Full Stack Web Development',
  'Credit_Hours__c': 12,
  'Boot_Camp_Course__c': 1},
 {'ID_Course__c': 3,
  'Course_Code__c': 'BC-UIUX',
  'Course_Name__c': 'User Interface/User Experience',
  'Credit_Hours__c': 12,
  'Boot_Camp_Course__c': 1},
 {'ID_Course__c': 4,
  'Course_Code__c': 'CIS-349',
  'Course_Name__c': 'Introduction to Databases',
  'Credit_Hours__c': 5,
  'Boot_Camp_Course__c': 0},
 {'ID_Course__c': 5,
  'Course_Code__c': 'CIS-405',
  'Course_Name__c': 'Database Programming',
  'Credit_Hours__c': 5,
  'Boot_Camp_Course__c': 0},
 {'ID_Course__c': 6,
  'Course_Code__c': 'CIS-438',
  'Course_Name__c': 'Database Administration',
  'Credit_Hours__c': 5,
  'Boot_Camp_Course__c': 0},
 {'ID_Course__c': 7,
  'Course_Code__c': 'CIS-430',
  'Course_

In [163]:
for rec in course_data_records:

    record = {
        'ID_Course__c': rec['ID_Course__c'],
        'Course_Code__c': rec['Course_Code__c'],
        'Course_Name__c': rec['Course_Name__c'],
        'Credit_Hours__c': rec['Credit_Hours__c'],
        'Boot_Camp_Course__c': rec['Boot_Camp_Course__c']
    }
    
    try:
        sf.Course__C.create(record)
    except Exception as e:
        print(e)
        

In [164]:
course_lookup_list = []
data = sf.query_all_iter("SELECT ID_Course__c, Name FROM Course__c")
for row in data:
    rec = {
        'ID_Course__c': row['ID_Course__c'],
        'Course__c': row['Name']
    }
    course_lookup_list.append(rec)

In [165]:
course_lookup_df = pd.DataFrame(course_lookup_list)
course_lookup_df

,ID_Course__c,Course__c
0,7.0,a0K3h000001ijTr
1,3.0,a0K3h000001ijTX
2,8.0,a0K3h000001ijTw
3,6.0,a0K3h000001ijTm
4,2.0,a0K3h000001ijTS
5,1.0,a0K3h000001ijTN
6,4.0,a0K3h000001ijTc
7,5.0,a0K3h000001ijTh


# 2) Class Table

In [191]:
#CLASS
query = '''
    SELECT
        c.*
        
    FROM
        class c
        
'''
class_data_df = pd.read_sql(query, conn)
class_data_df.head()

,ID_Class,ID_Course,Section,StartDate,EndDate
0,1,1,GWU-ARL-DATA-PT-09-0,2020-03-16,2020-03-14
1,2,1,GWDC201805DATA3,2018-05-15,2018-11-08
2,3,2,GWARL201905WEB3,2019-05-14,2019-11-07
3,4,3,GWARL201905UIUX3,2019-05-14,2019-11-07


In [192]:
class_data_df.rename(columns={
    'ID_Class' : 'ID_Class__c',
    'ID_Course': 'ID_Course__c',
    'Section':'Section__c',
    'StartDate':'Start_Date__c',
    'EndDate':'End_date__c'
}, inplace=True)
class_data_df

,ID_Class__c,ID_Course__c,Section__c,Start_Date__c,End_date__c
0,1,1,GWU-ARL-DATA-PT-09-0,2020-03-16,2020-03-14
1,2,1,GWDC201805DATA3,2018-05-15,2018-11-08
2,3,2,GWARL201905WEB3,2019-05-14,2019-11-07
3,4,3,GWARL201905UIUX3,2019-05-14,2019-11-07


In [193]:
class_data_df = pd.merge(class_data_df, course_lookup_df, how='left')
class_data_df.head()

,ID_Class__c,ID_Course__c,Section__c,Start_Date__c,End_date__c,Course__c
0,1,1,GWU-ARL-DATA-PT-09-0,2020-03-16,2020-03-14,a0K3h000001ijTN
1,2,1,GWDC201805DATA3,2018-05-15,2018-11-08,a0K3h000001ijTN
2,3,2,GWARL201905WEB3,2019-05-14,2019-11-07,a0K3h000001ijTS
3,4,3,GWARL201905UIUX3,2019-05-14,2019-11-07,a0K3h000001ijTX


In [194]:
class_data_df['Start_Date__c'] = pd.to_datetime(class_data_df['Start_Date__c']).dt.date
class_data_df['End_date__c'] = pd.to_datetime(class_data_df['End_date__c']).dt.date
class_data_df.head()

,ID_Class__c,ID_Course__c,Section__c,Start_Date__c,End_date__c,Course__c
0,1,1,GWU-ARL-DATA-PT-09-0,2020-03-16,2020-03-14,a0K3h000001ijTN
1,2,1,GWDC201805DATA3,2018-05-15,2018-11-08,a0K3h000001ijTN
2,3,2,GWARL201905WEB3,2019-05-14,2019-11-07,a0K3h000001ijTS
3,4,3,GWARL201905UIUX3,2019-05-14,2019-11-07,a0K3h000001ijTX


In [195]:
for index, row in class_data_df.iterrows():
        record = {
        'ID_Class__c': row['ID_Class__c'],
        'CourseID_FK__c': row['Course__c'],
        'Section__c': row['Section__c'],
        'Start_Date__c': str(row['Start_Date__c']),
        'End_date__c': str(row['End_date__c']),
    }
    
        try:
            sf.Class__c.create(record)
        except Exception as e:
            print(e)
    

In [196]:
class_lookup_list = []
data = sf.query_all_iter("SELECT ID_Class__c, Name FROM Class__c")
for row in data:
    rec = {
        'Class_ID__c': row['ID_Class__c'],
        'Class__c': row['Name']
    }
    class_lookup_list.append(rec)

class_lookup_df = pd.DataFrame(class_lookup_list)
class_lookup_df

,Class_ID__c,Class__c
0,1.0,a0L3h000001MXBj
1,4.0,a0L3h000001MXBy
2,3.0,a0L3h000001MXBt
3,2.0,a0L3h000001MXBo


# 3) Staff Table

In [200]:
staff_data_df = pd.read_sql("SELECT * FROM staff", conn)
staff_data_df.head(30)

,ID_Staff,EmployeeID,LastName,FirstName,MiddleName,BirthDate
0,1,000184220,Wimberly,Sam,Nico,1995-07-05
1,2,000130109,Sanford,Gemini,Blair,1992-04-22
2,3,000160655,Williams,Dartanion,De Angelo,1993-05-21
3,4,000159108,Popelka,Sarah,Nisan,1994-06-07


In [201]:
staff_data_df.rename(columns={
    'ID_Staff':'ID_Staff__c',
    'EmployeeID':'EmployeeID__c',
    'LastName':'LastName__c',
    'FirstName':'FirstName__c',
    'MiddleName':'MiddleName__c', 
    'BirthDate':'Date_of_Birth__c'
}, inplace=True)
staff_data_df.head()

,ID_Staff__c,EmployeeID__c,LastName__c,FirstName__c,MiddleName__c,Date_of_Birth__c
0,1,000184220,Wimberly,Sam,Nico,1995-07-05
1,2,000130109,Sanford,Gemini,Blair,1992-04-22
2,3,000160655,Williams,Dartanion,De Angelo,1993-05-21
3,4,000159108,Popelka,Sarah,Nisan,1994-06-07


In [202]:
staff_data_df['Date_of_Birth__c'] = staff_data_df['Date_of_Birth__c'].astype('str')
staff_data_df.head()

,ID_Staff__c,EmployeeID__c,LastName__c,FirstName__c,MiddleName__c,Date_of_Birth__c
0,1,000184220,Wimberly,Sam,Nico,1995-07-05
1,2,000130109,Sanford,Gemini,Blair,1992-04-22
2,3,000160655,Williams,Dartanion,De Angelo,1993-05-21
3,4,000159108,Popelka,Sarah,Nisan,1994-06-07


In [203]:
staff_data_records =staff_data_df.to_dict('records')
staff_data_records

[{'ID_Staff__c': 1,
  'EmployeeID__c': '000184220',
  'LastName__c': 'Wimberly',
  'FirstName__c': 'Sam',
  'MiddleName__c': 'Nico',
  'Date_of_Birth__c': '1995-07-05'},
 {'ID_Staff__c': 2,
  'EmployeeID__c': '000130109',
  'LastName__c': 'Sanford',
  'FirstName__c': 'Gemini',
  'MiddleName__c': 'Blair',
  'Date_of_Birth__c': '1992-04-22'},
 {'ID_Staff__c': 3,
  'EmployeeID__c': '000160655',
  'LastName__c': 'Williams',
  'FirstName__c': 'Dartanion',
  'MiddleName__c': 'De Angelo',
  'Date_of_Birth__c': '1993-05-21'},
 {'ID_Staff__c': 4,
  'EmployeeID__c': '000159108',
  'LastName__c': 'Popelka',
  'FirstName__c': 'Sarah',
  'MiddleName__c': 'Nisan',
  'Date_of_Birth__c': '1994-06-07'}]

In [204]:
for rec in staff_data_records:

    record = {
        'ID_Staff__c': rec['ID_Staff__c'],
        'EmployeeID__c': rec['EmployeeID__c'],
        'LastName__c': rec['LastName__c'],
        'FirstName__c': rec['FirstName__c'],
        'MiddleName__c': rec['MiddleName__c'],
        'Date_of_Birth__c': rec['Date_of_Birth__c']
    }
    
    try:
        sf.staff__c.create(record)
    except Exception as e:
        print(e)

In [205]:
staff_class_lookup_list = []
data = sf.query_all_iter("SELECT ID_Staff__c, Name FROM Staff__c")
for row in data:
    rec = {
        'ID_Staff__c': row['ID_Staff__c'],
        'Staff_Name__c': row['Name']
    }
    staff_class_lookup_list.append(rec)

In [206]:
staff_lookup_df = pd.DataFrame(staff_class_lookup_list)
staff_lookup_df

,ID_Staff__c,Staff_Name__c
0,2.0,a0M3h000002pGBi
1,3.0,a0M3h000002pGIn
2,4.0,a0M3h000002pGIs
3,1.0,a0M3h000002pGIi


# 4) Staff Assignment table

In [208]:
staffassignment_data_df = pd.read_sql("SELECT * FROM staffassignment", conn)
staffassignment_data_df.head(30)

,ID_StaffAssignment,ID_Staff,ID_Class,Role,StartDate,EndDate
0,1,1,1,Teacher Assistant,2020-03-16,None
1,2,4,1,Teacher Assistant,2020-03-16,None
2,3,3,1,Instructor,2020-03-16,None


In [209]:
staffassignment_data_df.rename(columns={
    'ID_StaffAssignment' : 'ID_StaffAssignment__c',
    'ID_Staff': 'ID_Staff__c',
    'ID_Class':'Class_ID__c',
    'Role':'Role__c',
    'StartDate':'StartDate__c',
    'EndDate':'EndDate__c'
}, inplace=True)
staffassignment_data_df

,ID_StaffAssignment__c,ID_Staff__c,Class_ID__c,Role__c,StartDate__c,EndDate__c
0,1,1,1,Teacher Assistant,2020-03-16,None
1,2,4,1,Teacher Assistant,2020-03-16,None
2,3,3,1,Instructor,2020-03-16,None


In [210]:
staffassignment_data_df = pd.merge(staffassignment_data_df, staff_lookup_df, how='left')
staffassignment_data_df.head()

,ID_StaffAssignment__c,ID_Staff__c,Class_ID__c,Role__c,StartDate__c,EndDate__c,Staff_Name__c
0,1,1,1,Teacher Assistant,2020-03-16,None,a0M3h000002pGIi
1,2,4,1,Teacher Assistant,2020-03-16,None,a0M3h000002pGIs
2,3,3,1,Instructor,2020-03-16,None,a0M3h000002pGIn


In [211]:
staffassignment_data_df = pd.merge(staffassignment_data_df, class_lookup_df, how='left')
staffassignment_data_df.head()

,ID_StaffAssignment__c,ID_Staff__c,Class_ID__c,Role__c,StartDate__c,EndDate__c,Staff_Name__c,Class__c
0,1,1,1,Teacher Assistant,2020-03-16,None,a0M3h000002pGIi,a0L3h000001MXBj
1,2,4,1,Teacher Assistant,2020-03-16,None,a0M3h000002pGIs,a0L3h000001MXBj
2,3,3,1,Instructor,2020-03-16,None,a0M3h000002pGIn,a0L3h000001MXBj


In [212]:
# staffassignment_data_df['StartDate__c'] = staffassignment_data_df['StartDate__c'].astype('str')
staffassignment_data_df['StartDate__c'] = pd.to_datetime(staffassignment_data_df['StartDate__c']).dt.date

staffassignment_data_df.head()

,ID_StaffAssignment__c,ID_Staff__c,Class_ID__c,Role__c,StartDate__c,EndDate__c,Staff_Name__c,Class__c
0,1,1,1,Teacher Assistant,2020-03-16,None,a0M3h000002pGIi,a0L3h000001MXBj
1,2,4,1,Teacher Assistant,2020-03-16,None,a0M3h000002pGIs,a0L3h000001MXBj
2,3,3,1,Instructor,2020-03-16,None,a0M3h000002pGIn,a0L3h000001MXBj


In [213]:
for index, row in staffassignment_data_df.iterrows():
        record = {
        'ID_StaffAssignment__c': row['ID_StaffAssignment__c'],
        'Staff_ID_FK__c': row['Staff_Name__c'],
        'Class_ID_FK__c': row['Class__c'],
        'Role__c': str(row['Role__c']),
        'StartDate__c': str(row['StartDate__c']),    
#         'EndDate__c': str(row['EndDate__c'])
    }
    
        try:
            sf.StaffAssignment__c.create(record)
        except Exception as e:
            print(e)

# 5) Student Table

In [214]:
student_data_df = pd.read_sql("SELECT * FROM student", conn)
student_data_df.head()

,ID_Student,StudentID,LastName,FirstName,MiddleName,BirthDate,Gender
0,33,25004961,Moore,Heather,Alice,None,F
1,34,25003514,Multak,Ilana,Cecille,None,F
2,35,25005833,Murillo,Jessica,Dorothy,None,F
3,36,25002589,Romanowski,Kandra,Genevieve,None,F
4,37,25007185,Hoffer,Katherine,Lynnette,None,F


In [215]:
student_data_df.rename(columns={
    'ID_Student' : 'IDStudent__c',
    'StudentID' : 'Student_ID__c',
    'LastName': 'Last_Name__c',
    'FirstName':'First_Name__c',
    'MiddleName':'Middle_Name__c',
    'BirthDate':'Date_of_Birth__c',
    'Gender':'Gender__c'
}, inplace=True)
student_data_df.head()

,IDStudent__c,Student_ID__c,Last_Name__c,First_Name__c,Middle_Name__c,Date_of_Birth__c,Gender__c
0,33,25004961,Moore,Heather,Alice,None,F
1,34,25003514,Multak,Ilana,Cecille,None,F
2,35,25005833,Murillo,Jessica,Dorothy,None,F
3,36,25002589,Romanowski,Kandra,Genevieve,None,F
4,37,25007185,Hoffer,Katherine,Lynnette,None,F


In [216]:
for index, row in student_data_df.iterrows():
        record = {
        'IDStudent__c': row['IDStudent__c'],
        'Student_ID__c': row['Student_ID__c'],
        'Last_Name__c': row['Last_Name__c'],
        'First_Name__c': row['First_Name__c'],
        'Middle_Name__c': str(row['Middle_Name__c']),
        'Gender__c': str(row['Gender__c'])    
    }
    
        try:
            sf.Student__c.create(record)
        except Exception as e:
            print(e)

In [226]:
student_lookup_list = []
data = sf.query_all_iter("SELECT IDStudent__c, Name FROM Student__c")
for row in data:
    rec = {
        'IDStudent__c': row['IDStudent__c'],
        'Student_ID_FK__c': row['Name']
    }
    student_lookup_list.append(rec)

In [227]:
student_lookup_df = pd.DataFrame(student_lookup_list)
student_lookup_df.head()

,IDStudent__c,Student_ID_FK__c
0,33.0,a0O3h0000014FPZ
1,34.0,a0O3h0000014FPe
2,35.0,a0O3h0000014FPj
3,36.0,a0O3h0000014FPo
4,37.0,a0O3h0000014FPt


# 6) Class Participant Table

In [248]:
classparticipant_data_df = pd.read_sql("SELECT * FROM classparticipant", conn)
classparticipant_data_df.head()

,ID_ClassParticipant,ID_Student,ID_Class,StartDate,EndDate
0,1,33,1,2020-03-16,None
1,2,34,1,2020-03-16,None
2,3,35,1,2020-03-16,None
3,4,62,1,2020-03-16,None
4,5,36,1,2020-03-16,None


In [249]:
classparticipant_data_df.rename(columns={
    'ID_ClassParticipant' : 'ID_ClassParticipant__c',
    'ID_Student' : 'IDStudent__c',
    'ID_Class': 'Class_ID__c',
    'StartDate':'Start_Date__c',
    'EndDate':'End_Date__c'
}, inplace=True)
classparticipant_data_df.head()

,ID_ClassParticipant__c,IDStudent__c,Class_ID__c,Start_Date__c,End_Date__c
0,1,33,1,2020-03-16,None
1,2,34,1,2020-03-16,None
2,3,35,1,2020-03-16,None
3,4,62,1,2020-03-16,None
4,5,36,1,2020-03-16,None


In [250]:
classparticipant_data_df = pd.merge(classparticipant_data_df, student_lookup_df, how='left')
classparticipant_data_df.head()

,ID_ClassParticipant__c,IDStudent__c,Class_ID__c,Start_Date__c,End_Date__c,Student_ID_FK__c
0,1,33,1,2020-03-16,None,a0O3h0000014FPZ
1,2,34,1,2020-03-16,None,a0O3h0000014FPe
2,3,35,1,2020-03-16,None,a0O3h0000014FPj
3,4,62,1,2020-03-16,None,a0O3h0000014FRu
4,5,36,1,2020-03-16,None,a0O3h0000014FPo


In [251]:
classparticipant_data_df = pd.merge(classparticipant_data_df, class_lookup_df, how='left')
classparticipant_data_df.head()

,ID_ClassParticipant__c,IDStudent__c,Class_ID__c,Start_Date__c,End_Date__c,Student_ID_FK__c,Class__c
0,1,33,1,2020-03-16,None,a0O3h0000014FPZ,a0L3h000001MXBj
1,2,34,1,2020-03-16,None,a0O3h0000014FPe,a0L3h000001MXBj
2,3,35,1,2020-03-16,None,a0O3h0000014FPj,a0L3h000001MXBj
3,4,62,1,2020-03-16,None,a0O3h0000014FRu,a0L3h000001MXBj
4,5,36,1,2020-03-16,None,a0O3h0000014FPo,a0L3h000001MXBj


In [254]:
classparticipant_data_df['Start_Date__c'] = classparticipant_data_df['Start_Date__c'].astype('str')
classparticipant_data_df['End_Date__c'] = classparticipant_data_df['End_Date__c'].astype('str')
# classparticipant_data_df['Start_Date__c'] = pd.to_datetime(classparticipant_data_df['Start_Date__c']).dt.date
# classparticipant_data_df['End_Date__c'] = pd.to_datetime(classparticipant_data_df['End_Date__c']).dt.date

classparticipant_data_df.head()

,ID_ClassParticipant__c,IDStudent__c,Class_ID__c,Start_Date__c,End_Date__c,Student_ID_FK__c,Class__c
0,1,33,1,2020-03-16,None,a0O3h0000014FPZ,a0L3h000001MXBj
1,2,34,1,2020-03-16,None,a0O3h0000014FPe,a0L3h000001MXBj
2,3,35,1,2020-03-16,None,a0O3h0000014FPj,a0L3h000001MXBj
3,4,62,1,2020-03-16,None,a0O3h0000014FRu,a0L3h000001MXBj
4,5,36,1,2020-03-16,None,a0O3h0000014FPo,a0L3h000001MXBj


In [259]:
for index, row in classparticipant_data_df.iterrows():
        record = {
        'ID_ClassParticipant__c': row['ID_ClassParticipant__c'],
        'Student_ID_FK__c': row['Student_ID_FK__c'],
        'Class_ID_FK__c': row['Class__c'],
        'Start_Date__c': row['Start_Date__c'],
#         'End_Date__c': str(row['End_Date__c'])
        }
    
        try:
            sf.Class_Participant__c.create(record)
        except Exception as e:
            print(e)

# Bulk Delete Rocords

In [ ]:

# course_records = sf.query("SELECT Id FROM student__c")
# recs_to_delete = [{'Id': r['Id']} for r in course_records['records']]
# recs_to_delete
# sf.bulk.student__c.delete(recs_to_delete)